## Reed-Muller Code:

In [2]:
import numpy as np
import itertools

class RMCode:
    
    def __init__(self, r, m):
            
        self.r = r
        self.m = m
        self.q = 2
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        self.message_type = 'bin'
        
        # Initializing field
        self.F = FiniteField(self.q)
        
        self.G = matrix(self.F, self.k, self.n, self.generateG(self.r, self.m))
        
        
    def generateG(self, r, m):
        ones = np.repeat(1, 2**m)
        bases = [np.array([(i // 2**a) % 2 for i in range(2**m)]) for a in range(m)]
        if r == 1:
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases)])
            return G
        elif r > 1:
            others = []
            for i in range(2,r+1):
                others.extend([math.prod(x) for x in itertools.combinations(bases, i)])
            
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases), np.stack(others)])
            return G
        else:
            raise ValueError('wrong value of r')
    
    
    def Encoding(self, message, zeropad = True, out = 'bin', concatenated_dimension = None):
        
        if not concatenated_dimension:
            concatenated_dimension = self.k
        else:
            concatenated_dimension = concatenated_dimension
        
        data_type = self._DetermineInput(message)
        
        if (data_type == 'pol' or data_type == 'bin'):
            message = list(message)
        elif data_type == 'int':
            pass
        else:
            raise ValueError('Wrong data type')
        
        rem = len(message) % concatenated_dimension
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(concatenated_dimension-rem))
            else:
                raise ValueError('k does not divide input size')    
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        if out == 'pol':
            return vector(self.F, c)
        elif out == 'int':
            c = self._PolToInt(c)
            return c
        elif out == 'bin':
            c = self._PolToInt(c)
            c = self._IntToBitString(c)
            return c
        else:
            raise ValueError('Unrecognized output')
        
            
    def EncodeChunk(self, chunk):
        
        #chunk = list(chunk)
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received, out = 'bin'):
        
        #received = vector(self.F, received)
        data_type = self._DetermineInput(received)
        
        if data_type == 'pol':
            received = vector(self.F, received)
        elif data_type == 'bin':
            received = vector(self.F, received)
        elif data_type == 'int':
            received = vector(self.F, received)
        else:
            raise ValueError('Wrong data type')
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        d = []
        
        for i in range(0,len(received),self.n):
            d.extend(self.DecodeChunk(received[i:i+self.n]))
            
            
        if out == 'pol':
            return vector(self.F, d)
        elif out == 'int':
            d = self._PolToInt(d)
            return d
        elif out == 'bin':
            d = self._PolToInt(d)
            d = self._IntToBitString(d)
            return d
        else:
            raise ValueError('Unrecognized output')
                
            
    def DecodeChunk(self, word):
        # Reed Decoding algorithm
        
        if (len(word) != self.n):
            raise ValueError('Invalid chunk size')
        
        decoded = []
        
        variables = [i for i in range(1, self.m + 1)]
        r = self.r
        G = self.G
        
        while (r > 0):
            current_variables = []
            current_variables.append(list(itertools.combinations(variables,r)))
            current_variables = list(current_variables)
            
            for i in range(binomial(self.m,r)-1, -1, -1):

                # compute the complementary set T
                T = [i for i in range(1,self.m+1)]
                for j in range(r):
                    T.remove(current_variables[0][i][j])
                    
                combinations_T = self.DecodingConvert(T)
                combinations_S = self.DecodingConvert(list(current_variables[0][i]))
                votings = []
                for i in range(len(combinations_T)):
                    voting = 0
                    for j in range(len(combinations_S)):
                        voting += word[combinations_T[i] + combinations_S[j]]

                    votings.append(voting)
                decoded = self.MajorityDecoding(votings) + decoded
                
            G_part = matrix(self.F, binomial(self.m,r), self.n, G[-binomial(self.m,r):][:])
            word_part = vector(self.F, decoded[:binomial(self.m,r)]) * G_part
            word += word_part
            
            G = G[:-binomial(self.m,r)][:]
            
            r -= 1
                
        # decode the first coefficient
        decoded = self.MajorityDecoding(word.list()) + decoded
        
        return vector(self.F, decoded)
    
    
    def DecodingConvert(self, elements): # takes as input one tuple
        combinations = []
        for i in range(len(elements)+1):
            combinations.append(list(itertools.combinations(elements,i)))
        combinations = list(combinations)
        integers = []
        for i in range(len(combinations)):
            for j in range(len(combinations[i])):
                l = [0 for i in range(self.m)]
                for k in range(len(combinations[i][j])):
                    l[combinations[i][j][k] - 1] = 1
                integers.append(ZZ(l,2))
        return integers

            
    def MajorityDecoding(self, word):
        # Decode by comparing the number of ones with the number of zeros
        
        if len(word) == 1:
            return word
        elif word.count(1) >= floor(len(word) / 2):
            return [1]
        elif word.count(1) < floor(len(word) / 2):
            return [0]
        else:
            return "decoding failure"
        
        
    def _DetermineInput(self, data):
        # determine data type

        if isinstance(data, str) and all([(bit == '1' or bit == '0') for bit in data]):

            return('bin')

        if data[0] == None:
            return('none')
        elif data[0].parent() == self.F:
            return('pol')
        elif data[0].parent() == ZZ:
            return('int')
        else:
            return('unknown')

    def _PolToInt(self, pol_array):

        # convert array of polynomial representation to array of integers

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.q))

        return(pol_out)
    
    def _IntToBitString(self, int_array):

        # Converts array of integers less than 2 to bit string

        if any([(item > (self.q - 1) or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return(''.join([format(item, '01b') for item in int_array]))

In [3]:
#C = RMCode(r =1, m = 3)
#C.k

In [4]:
#m = vector(GF(2), [1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0])
#m = [1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1]
#m = '11001100110011001'

#c = C.Encoding(m, out = 'pol')
#c = C.Encoding(m, out = 'int')
#c = C.Encoding(m, out = 'bin')
#print('codeword: ', c)

#d = C.Decoding(c, out = 'pol')
#d = C.Decoding(c, out = 'int')
#d = C.Decoding(c, out = 'bin')
#print('decoded word: ', d)
